Your goal is to achieve a regression model with a consistent R2 and only statistically significant parameters across multiple samples.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn import linear_model
from sklearn.feature_selection import f_regression
import statsmodels.formula.api as smf
import ny_crime2013 as ny

## New York Crime Data

In [2]:
Y = ny.crime_model["Property Crime"]
X = ny.crime_model[["Population","Murder","Robbery","Population^2"]]
ny.crime_regr.fit(X,Y)

print("Coefficients:\n",ny.crime_regr.coef_)
print("\nIntercept:\n",ny.crime_regr.intercept_)
print("\nR-squared: ")
print(ny.crime_regr.score(X,Y))

Coefficients:
 [2.39839687e-01 5.48217901e-06 1.03217668e-01 1.25644391e-01]

Intercept:
 -0.9079588526233651

R-squared: 
0.8150006601593579


In [3]:
# test entire model for R2 significance
f_val,p_val = f_regression(X,Y)
print("F-values {}".format(f_val))
print("p-values {}".format(p_val))

F-values [1013.0186961   109.69065863  648.05620139 1034.83628683]
p-values [1.10531723e-104 1.80059421e-022 3.31544037e-081 7.05207608e-106]


Appears that the entire model can predict an outcome, R2 value is significant. 

In [4]:
# test individual features for performance
ny.crime_model = ny.crime_model.rename(columns={'Property Crime':'PropertyCrime','Population^2':'Population2'})
linear_formula = "PropertyCrime~Population+Murder+Robbery+Population2"
lm = smf.ols(formula=linear_formula,data=ny.crime_model).fit()

In [5]:
lm.params

Intercept     -0.907959
Population     0.239840
Murder         0.000005
Robbery        0.103218
Population2    0.125644
dtype: float64

In [6]:
lm.pvalues

Intercept      5.385646e-17
Population     1.729098e-01
Murder         1.075840e-01
Robbery        3.479628e-21
Population2    4.028069e-02
dtype: float64

In [7]:
lm.conf_int()

,0,1
Intercept,-1.110124,-0.705793
Population,-0.105568,0.585248
Murder,-0.000001,0.000012
Robbery,0.083130,0.123306
Population2,0.005601,0.245688


In [8]:
lm.rsquared

0.8150006601593579

## California Crime Data (Validating New York Regression Model)

In [9]:
# test accuracy of model using California crime dataset
ca_data = pd.read_excel("table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls",skiprows=4)
ca_data.head(5)

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,Adelanto,31165.0,198.0,2.0,NaN,15.0,52.0,129.0,886.0,381.0,372.0,133.0,17.0
1,Agoura Hills,20762.0,19.0,0.0,NaN,2.0,10.0,7.0,306.0,109.0,185.0,12.0,7.0
2,Alameda,76206.0,158.0,0.0,NaN,10.0,85.0,63.0,1902.0,287.0,1285.0,330.0,17.0
3,Albany,19104.0,29.0,0.0,NaN,1.0,24.0,4.0,557.0,94.0,388.0,75.0,7.0
4,Alhambra,84710.0,163.0,1.0,NaN,9.0,81.0,72.0,1774.0,344.0,1196.0,234.0,7.0


In [10]:
ca_data["Population2"] = np.square(ca_data["Population"])

In [11]:
ca_data.isnull().sum()

City                                        0
Population                                  2
Violent\ncrime                              2
Murder and\nnonnegligent\nmanslaughter      2
Rape\n(revised\ndefinition)1              464
Rape\n(legacy\ndefinition)2                 2
Robbery                                     2
Aggravated\nassault                         2
Property\ncrime                             2
Burglary                                    2
Larceny-\ntheft                             2
Motor\nvehicle\ntheft                       2
Arson                                       2
Population2                                 2
dtype: int64

In [12]:
ca_data = ca_data.rename(columns={"Property\ncrime":"Property Crime","Murder and\nnonnegligent\nmanslaughter":"Murder"})

In [13]:
col_list = ["Property Crime","Population","Murder","Robbery","Population2"]
ca_crime = pd.DataFrame(columns=col_list)
for col in col_list:
    ca_crime[col] = ca_data[col]
ca_crime.head(5)

,Property Crime,Population,Murder,Robbery,Population2
0,886.0,31165.0,2.0,52.0,9.712572e+08
1,306.0,20762.0,0.0,10.0,4.310606e+08
2,1902.0,76206.0,0.0,85.0,5.807354e+09
3,557.0,19104.0,0.0,24.0,3.649628e+08
4,1774.0,84710.0,1.0,81.0,7.175784e+09


In [14]:
ca_crime.isnull().sum()

Property Crime    2
Population        2
Murder            2
Robbery           2
Population2       2
dtype: int64

In [15]:
ca_crime = ca_crime.dropna(how="all")
ca_crime.isnull().sum()

Property Crime    0
Population        0
Murder            0
Robbery           0
Population2       0
dtype: int64

## Validate NY Regression Model using CA data

In [16]:
Y2 = ca_crime["Property Crime"]
X2 = ca_crime[["Population","Murder","Robbery","Population2"]]
cacrime_regr = linear_model.LinearRegression()
cacrime_regr.fit(X2,Y2)
print("Coefficients:\n",cacrime_regr.coef_)
print("\nIntercept:\n",cacrime_regr.intercept_)
print("\nR-squared: ")
print(cacrime_regr.score(X2,Y2))

Coefficients:
 [ 2.43162820e-02 -2.55207539e+01  5.44672640e+00 -2.97874436e-09]

Intercept:
 -132.53343034154477

R-squared: 
0.9713660340651297


R2 value appears to be extremely high for CA data, may be sign of overfitting. Will attempt to apply cross validation.

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X2,Y2,test_size=0.3,random_state=20)
print("With 30% Holdout: " + str(cacrime_regr.fit(X_train,y_train).score(X_test,y_test)))
print("Testing on Sample: " + str(cacrime_regr.fit(X2,Y2).score(X2,Y2)))

With 30% Holdout: 0.8689123792579931
Testing on Sample: 0.9713660340651297


In [18]:
from sklearn.model_selection import cross_val_score
cross_val_score(cacrime_regr,X2,Y2,cv=10)

array([0.90357622, 0.85401115, 0.76939114, 0.85637489, 0.86123624,
       0.98387317, 0.15836641, 0.90909367, 0.84438614, 0.86853378])

In [19]:
from sklearn import metrics
Y2_predict = cacrime_regr.predict(X2)
accuracy = metrics.r2_score(Y2,Y2_predict)
print("Cross-Predicted Accuracy: ",accuracy)

Cross-Predicted Accuracy:  0.9713660340651297


## F-test for CA regression model and T-test features 

In [20]:
f_val,p_val = f_regression(X2,Y2)
print("F-values {}".format(f_val))
print("p-values {}".format(p_val))

F-values [3602.25899303 2282.03165776 3143.29555902  761.79011449]
p-values [1.03288688e-219 1.93930300e-180 9.84401798e-208 1.25147348e-099]


In [22]:
ca_crime = ca_crime.rename(columns={"Property Crime":"PropertyCrime"})
lm = smf.ols(formula=linear_formula,data=ca_crime).fit()

In [23]:
lm.params

Intercept     -1.325341e+02
Population     2.431629e-02
Murder        -2.552068e+01
Robbery        5.446725e+00
Population2   -2.978748e-09
dtype: float64

In [24]:
lm.pvalues

Intercept       1.157538e-02
Population     2.131235e-139
Murder          3.652474e-02
Robbery         5.890408e-70
Population2     2.098127e-57
dtype: float64

In [25]:
lm.conf_int()

,0,1
Intercept,-2.352741e+02,-2.979404e+01
Population,2.302343e-02,2.560916e-02
Murder,-4.943467e+01,-1.606684e+00
Robbery,4.941790e+00,5.951660e+00
Population2,-3.295313e-09,-2.662184e-09


In [26]:
lm.rsquared

0.9713660340651649

## The p-value for Population and Murder appear to be greater than significance level (0.05), will try to drop these features to revise model. 

In [21]:
Y = ny.crime_model["PropertyCrime"]
X = ny.crime_model[["Robbery","Population2"]]
ny.crime_regr.fit(X,Y)
print("Coefficients:\n",ny.crime_regr.coef_)
print("\nIntercept:\n",ny.crime_regr.intercept_)
print("\nR-squared: ")
print(ny.crime_regr.score(X,Y))

Coefficients:
 [0.10713909 0.21173059]

Intercept:
 -1.054280330833529

R-squared: 
0.8123270777813576


Rsquared value seems to have decreased slightly as a result of dropping Population and Murder Features (not signficantly adverse?)